In [0]:
%sh pip install dumps
pip install kafka-python
pip install tweepy

Collecting dumps
 Could not find a version that satisfies the requirement dumps (from versions: )
No matching distribution found for dumps
You are using pip version 19.0.3, however version 20.3b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already satisfied: kafka-python in /databricks/python3/lib/python3.7/site-packages (2.0.2)
You are using pip version 19.0.3, however version 20.3b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Collecting tweepy
 Downloading https://files.pythonhosted.org/packages/bb/7c/99d51f80f3b77b107ebae2634108717362c059a41384a1810d13e2429a81/tweepy-3.9.0-py2.py3-none-any.whl
Collecting requests-oauthlib>=0.7.0 (from tweepy)
 Downloading https://files.pythonhosted.org/packages/a3/12/b92740d845ab62ea4edf04d2f4164d82532b5a0b03836d4d4e71c6f3d379/requests_oauthlib-1.3.0-py2.py3-none-any.whl
Requirement already satisfied: six>=1.10.0 in /databricks/python3/lib/python3.7/site-packages (from tweepy) (1.12.0)
Requirement already satisfied: requests[socks]>=2.11.1 in /databricks/python3/lib/python3.7/site-packages (from tweepy) (2.21.0)
Collecting oauthlib>=3.0.0 (from requests-oauthlib>=0.7.0->tweepy)
 Downloading https://files.pythonhosted.org/packages/05/57/ce2e7a8fa7c0afb54a0581b14a65b56e62b5759dbc98e80627142b8a3704/oauthlib-3.1.0-py2.py3-none-any.whl (147kB)
Requirement already satisfied: chardet<3.1.0,>=3.0.2 in /databricks/python3/lib/python3.7/site-packages (from requests[socks]>=2.11.1->tweepy) (3.0.4)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.7/site-packages (from requests[socks]>=2.11.1->tweepy) (2019.3.9)
Requirement already satisfied: urllib3<1.25,>=1.21.1 in /databricks/python3/lib/python3.7/site-packages (from requests[socks]>=2.11.1->tweepy) (1.24.1)
Requirement already satisfied: idna<2.9,>=2.5 in /databricks/python3/lib/python3.7/site-packages (from requests[socks]>=2.11.1->tweepy) (2.8)
Requirement already satisfied: PySocks!=1.5.7,>=1.5.6; extra == "socks" in /databricks/python3/lib/python3.7/site-packages (from requests[socks]>=2.11.1->tweepy) (1.6.8)
Installing collected packages: oauthlib, requests-oauthlib, tweepy
Successfully installed oauthlib-3.1.0 requests-oauthlib-1.3.0 tweepy-3.9.0
You are using pip version 19.0.3, however version 20.3b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [0]:
from kafka import KafkaProducer
import kafka
import json
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
import sys
from json import dumps

producer = KafkaProducer(bootstrap_servers=['localhost:9092'], api_version=(0, 10),linger_ms=10)
# producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
#                          value_serializer=lambda x: 
#                          dumps(x).encode('utf-8'))

class StreamListener(tweepy.StreamListener):
    def on_status(self, status):
        # Check if it's not retweeted
        if hasattr(status, "retweeted_status"):
            # Do nothing if its retweeted
            pass
        else:
            # check if text has been truncated
            if hasattr(status, "extended_tweet"):
                text = status.extended_tweet["full_text"]
            else:
                text = status.text
            #prod = connect_kafka_producer()
            producer.send('numtest', value=text)
            #publish_message(prod, 'tweets', text)

    def on_error(self, status_code):
        print("Encountered streaming error with code ", status_code, "\n")

consumerKey = "gjQwas8CVDDogQzRRdHOvBomt"
consumerSecret = "jkvXugT9H5nK9Sxk1UCg65FtbqmkJZrDfSyoWA4VsJbKIOEe0E"
accessToken = "705048940933046272-LnQY1Az6el5PDQZIft22GDuGXsTEiH9"
accessTokenSecret = "nIUHdBjdV3WkcmWuVoK6ARGlt1Njjj9VZNbwnCY2Ogfvh"
# Complete Authorization and initialize endpoints
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)
# initialize stream

streamListener = StreamListener()
stream = tweepy.Stream(auth=api.auth, listener=streamListener, tweet_mode='extended')
stream.filter(languages=["en"], track=["#donald", "#corona", "2020", "#election", "#2020"])